In [ ]:
# Phase 1: Environment Setup and Data Loading

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os
import joblib
from datetime import datetime

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Fashion-MNIST class names
CLASS_NAMES = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]

print("Environment setup complete!")
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")
print(f"Number of classes: {len(CLASS_NAMES)}")


In [ ]:
# Load Fashion-MNIST Dataset

# Define data transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load training and test datasets
train_dataset = torchvision.datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

test_dataset = torchvision.datasets.FashionMNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

# Create data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")
print(f"Batch size: {batch_size}")
print(f"Training batches: {len(train_loader)}")
print(f"Test batches: {len(test_loader)}")

# Display sample images
fig, axes = plt.subplots(2, 5, figsize=(12, 6))
for i in range(10):
    image, label = train_dataset[i]
    axes[i//5, i%5].imshow(image.squeeze(), cmap='gray')
    axes[i//5, i%5].set_title(f'{CLASS_NAMES[label]}')
    axes[i//5, i%5].axis('off')
plt.tight_layout()
plt.show()


In [ ]:
# Define CNN Architecture
class FashionCNN(nn.Module):
    def __init__(self):
        super(FashionCNN, self).__init__()
        # First convolutional block
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        # Second convolutional block
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        # Third convolutional block
        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.conv6 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn6 = nn.BatchNorm2d(512)
        self.pool3 = nn.MaxPool2d(2, 2)
        
        # Fully connected layers
        self.fc1 = nn.Linear(512 * 3 * 3, 1024)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(1024, 512)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(512, 10)
        
    def forward(self, x):
        # First block
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        
        # Second block
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)
        
        # Third block
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.pool3(x)
        
        # Flatten and fully connected
        x = x.view(-1, 512 * 3 * 3)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        
        return x

# Initialize model
model = FashionCNN().to(device)

# Model summary
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Model architecture: 6-layer CNN with {len(list(model.modules()))} modules")


In [ ]:
# Training Setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

# Training loop
num_epochs = 3
train_losses = []
train_accuracies = []

print("Starting training...")
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
    
    for batch_idx, (data, target) in enumerate(progress_bar):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()
        
        # Update progress bar
        progress_bar.set_postfix({
            'Loss': f'{loss.item():.4f}',
            'Acc': f'{100.*correct/total:.2f}%'
        })
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_acc)
    
    scheduler.step()
    
    print(f'Epoch {epoch+1}: Loss = {epoch_loss:.4f}, Accuracy = {epoch_acc:.2f}%')

print("Training completed!")


In [ ]:
# Model Evaluation
model.eval()
test_correct = 0
test_total = 0
test_loss = 0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        loss = criterion(outputs, target)
        test_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        test_total += target.size(0)
        test_correct += (predicted == target).sum().item()
        
        # Calculate per-class accuracy
        c = (predicted == target).squeeze()
        for i in range(target.size(0)):
            label = target[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

test_accuracy = 100 * test_correct / test_total
test_loss = test_loss / len(test_loader)

print(f'Test Accuracy: {test_accuracy:.2f}%')
print(f'Test Loss: {test_loss:.4f}')
print('\nPer-class accuracy:')
for i in range(10):
    if class_total[i] > 0:
        print(f'{CLASS_NAMES[i]}: {100 * class_correct[i] / class_total[i]:.1f}%')

# Save the model
os.makedirs('models', exist_ok=True)
model_path = 'models/fashion_cnn_model.pt'
torch.save(model.state_dict(), model_path)
print(f'\nModel saved to {model_path}')

# Plot training curves
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses)
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.subplot(1, 2, 2)
plt.plot(train_accuracies)
plt.title('Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.tight_layout()
plt.show()


In [ ]:
# Test the API locally
# First, run the API server: uvicorn app:app --host 0.0.0.0 --port 8000

import requests
import json
from PIL import Image
import io

def test_api_health():
    """Test the health endpoint"""
    try:
        response = requests.get('http://localhost:8000/health')
        print(f"Health check status: {response.status_code}")
        print(f"Response: {response.json()}")
        return response.status_code == 200
    except Exception as e:
        print(f"API not running: {e}")
        return False

def test_api_prediction(image_path):
    """Test image prediction endpoint"""
    try:
        with open(image_path, 'rb') as f:
            files = {'file': f}
            response = requests.post('http://localhost:8000/predict_image', files=files)
        
        print(f"Prediction status: {response.status_code}")
        if response.status_code == 200:
            result = response.json()
            print(f"Prediction: {result['prediction']}")
            print(f"Confidence: {result['confidence']:.4f}")
            return result
        else:
            print(f"Error: {response.text}")
            return None
    except Exception as e:
        print(f"Prediction test failed: {e}")
        return None

# To test the API, run:
# 1. Start the API server in terminal: uvicorn app:app --host 0.0.0.0 --port 8000
# 2. Then uncomment and run the following lines:

# print("Testing API...")
# if test_api_health():
#     print("API is running successfully!")
#     # Test with a sample image
#     test_api_prediction('sample_images/test_pattern_1.png')
# else:
#     print("API is not running. Start it with: uvicorn app:app --host 0.0.0.0 --port 8000")

print("API testing functions ready!")
print("To test: Start API server and uncomment the test code above.")


In [ ]:
# Docker Container Commands
# Run these commands in your terminal:

docker_commands = """
# Build the Docker image
docker build -t fashion-cnn-api .

# Run the container locally
docker run -p 8000:8000 fashion-cnn-api

# Or use docker-compose (recommended)
docker-compose up --build

# Test the containerized API
curl http://localhost:8000/health

# Stop the container
docker-compose down
"""

print("Docker Commands for Container Build and Test:")
print(docker_commands)

# Create sample images for testing
import subprocess
import os

def create_test_images():
    """Create sample test images if they don't exist"""
    if not os.path.exists('sample_images'):
        print("Creating sample images...")
        try:
            subprocess.run(['python', 'create_sample_images.py'], check=True)
            print("Sample images created successfully!")
        except subprocess.CalledProcessError:
            print("Failed to create sample images. Create them manually.")
    else:
        print("Sample images directory already exists.")

def test_docker_container():
    """Test the Docker container endpoints"""
    commands = [
        "# Test health endpoint",
        "curl -X GET http://localhost:8000/health",
        "",
        "# Test prediction endpoint with sample image",
        "curl -X POST http://localhost:8000/predict_image -F 'file=@sample_images/test_pattern_1.png'",
        "",
        "# View API documentation",
        "curl -X GET http://localhost:8000/docs"
    ]
    
    print("Container Test Commands:")
    for cmd in commands:
        print(cmd)

create_test_images()
test_docker_container()

print("\nContainer deployment instructions:")
print("1. Build: docker build -t fashion-cnn-api .")
print("2. Run: docker run -p 8000:8000 fashion-cnn-api")
print("3. Test: curl http://localhost:8000/health")
print("4. Stop: docker stop <container_id>")


In [ ]:
# Cloud Deployment Commands
# Replace YOUR_SERVER_IP with your actual Hetzner server IP

deployment_commands = """
# 1. Copy files to server
scp -r . username@YOUR_SERVER_IP:/home/username/fashion-cnn-api/

# 2. SSH into server
ssh username@YOUR_SERVER_IP

# 3. Navigate to project directory
cd /home/username/fashion-cnn-api/

# 4. Build and run with docker-compose
docker-compose up --build -d

# 5. Check container status
docker ps

# 6. View logs
docker logs fashion_cnn_classifier

# 7. Test the deployed API
curl http://YOUR_SERVER_IP:8000/health
"""

print("Cloud Deployment Commands:")
print(deployment_commands)

# Production API Testing
def test_production_api(server_ip):
    """Test the production API deployment"""
    import requests
    
    base_url = f"http://{server_ip}:8000"
    
    try:
        # Test health endpoint
        response = requests.get(f"{base_url}/health")
        print(f"Health check: {response.status_code}")
        print(f"Response: {response.json()}")
        
        # Test prediction endpoint
        with open('sample_images/test_pattern_1.png', 'rb') as f:
            files = {'file': f}
            response = requests.post(f"{base_url}/predict_image", files=files)
        
        print(f"Prediction test: {response.status_code}")
        if response.status_code == 200:
            result = response.json()
            print(f"Prediction: {result['prediction']}")
            print(f"Confidence: {result['confidence']:.4f}")
        
        return True
    except Exception as e:
        print(f"Production API test failed: {e}")
        return False

# Monitoring and troubleshooting commands
monitoring_commands = """
# Monitor container performance
docker stats fashion_cnn_classifier

# Check container logs
docker logs -f fashion_cnn_classifier

# Restart container if needed
docker-compose restart

# Update deployment
docker-compose pull
docker-compose up --build -d

# Cleanup old images
docker image prune -f
"""

print("\nProduction Monitoring Commands:")
print(monitoring_commands)

# Final summary
print("\n=== DEPLOYMENT COMPLETE ===")
print("Your Fashion-MNIST CNN classifier is now deployed!")
print("API Endpoints:")
print("- Health: http://YOUR_SERVER_IP:8000/health")
print("- Predict: http://YOUR_SERVER_IP:8000/predict_image")
print("- Docs: http://YOUR_SERVER_IP:8000/docs")
print("\nTo test production API, update YOUR_SERVER_IP and run:")
print("test_production_api('YOUR_SERVER_IP')")

print("\n🎉 Project Complete! 🎉")
print("You've successfully:")
print("✅ Trained a CNN model")
print("✅ Created a FastAPI service")
print("✅ Containerized the application")
print("✅ Deployed to cloud infrastructure")
print("✅ Tested the production API")
